<a href="https://colab.research.google.com/github/hyrule-coder/langchain-book-learning/blob/main/chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. LangChain Expression Language (LCEL) 徹底解説

In [ ]:
!pip install langchain-openai==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "agent-book"
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


## 5.1 RunnableとRunnableSequence -- LCELの最も基本的な構成要素

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}")
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

#prompt_value = prompt.invoke({"dish": "カレー"})
#ai_message = model.invoke(prompt_value)
#output = output_parser.invoke(ai_message)

chain = prompt | model | output_parser
output = chain.invoke({"dish": "カレー"})

print(output)

カレーのレシピをご紹介します。シンプルで美味しい基本的なチキンカレーの作り方です。

### 材料（4人分）
- 鶏もも肉：400g（大きめの一口大にカット）
- 玉ねぎ：2個（みじん切り）
- にんにく：2片（みじん切り）
- 生姜：1片（みじん切り）
- トマト：1個（ざく切り）
- カレーパウダー：大さじ2
- 塩：小さじ1（お好みで調整）
- ココナッツミルク：200ml（または水）
- サラダ油：大さじ2
- パクチー（お好みで）：適量

### 作り方
1. **下準備**: 鶏肉は一口大にカットし、塩を振っておきます。玉ねぎ、にんにく、生姜、トマトはそれぞれみじん切り、ざく切りにします。

2. **玉ねぎを炒める**: 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを加え、中火で透明になるまで炒めます。

3. **香味野菜を加える**: 玉ねぎが透明になったら、にんにくと生姜を加え、香りが立つまでさらに炒めます。

4. **鶏肉を加える**: 鶏もも肉を鍋に加え、表面が白くなるまで炒めます。

5. **トマトとスパイスを加える**: ざく切りにしたトマトとカレーパウダーを加え、全体をよく混ぜます。トマトが崩れるまで炒めます。

6. **煮込む**: ココナッツミルク（または水）を加え、全体を混ぜたら、蓋をして中火で約15分煮込みます。時々かき混ぜて、焦げ付かないように注意します。

7. **味を調える**: 煮込みが終わったら、塩で味を調整します。お好みで辛さを足したい場合は、チリパウダーやカイエンペッパーを加えても良いです。

8. **盛り付け**: お皿に盛り付け、お好みでパクチーを散らして完成です。

### 提供方法
ご飯やナンと一緒に楽しんでください。サラダやヨーグルトを添えると、より一層美味しくいただけます。

お好みで野菜（じゃがいも、にんじん、ピーマンなど）を加えても美味しいです。お試しください！


### Runnableの実行方法 --invoke・stream・batch

#### Stream

In [ ]:
chain = prompt | model | output_parser
for chunk in chain.stream({"dish": "カレー"}):
    print(chunk, end="", flush=True)

カレーのレシピをご紹介します！以下は基本的なチキンカレーのレシピです。

### 材料（4人分）
- 鶏もも肉：400g（食べやすい大きさにカット）
- 玉ねぎ：2個（みじん切り）
- にんにく：2片（みじん切り）
- 生姜：1片（みじん切り）
- トマト：1個（ざく切り）
- カレーパウダー：大さじ2
- クミンシード：小さじ1（お好みで）
- ココナッツミルク：200ml（お好みで）
- サラダ油：大さじ2
- 塩：適量
- 黒胡椒：適量
- 水：400ml
- パクチー（飾り用）：適量

### 作り方
1. **下準備**: 鶏肉に塩と黒胡椒をふりかけて下味をつけておきます。

2. **玉ねぎを炒める**: 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを加え、中火で透明になるまで炒めます。

3. **香味野菜を加える**: にんにくと生姜を加え、香りが立つまでさらに炒めます。

4. **スパイスを加える**: クミンシードを加え、さらに1分ほど炒めた後、カレーパウダーを加え、全体がよく混ざるまで炒めます。

5. **鶏肉を加える**: 鶏肉を鍋に加え、表面が白くなるまで炒めます。

6. **トマトと水を加える**: ざく切りにしたトマトと水を加え、全体をよく混ぜます。沸騰したら、弱火にして蓋をし、約20分煮込みます。

7. **ココナッツミルクを加える**: 煮込みが終わったら、ココナッツミルクを加え、さらに5分ほど煮ます。味を見て、必要に応じて塩で調整します。

8. **盛り付け**: お皿に盛り付け、パクチーを散らして完成です。

### 提供方法
ご飯やナンと一緒にお召し上がりください。お好みでヨーグルトやサラダを添えると、より美味しく楽しめます。

ぜひお試しください！

#### batch

In [ ]:
chain = prompt | model | output_parser

outputs = chain.batch([{"dish":"カレー"}, {"dish":"うどん"}])
print(outputs)

['カレーのレシピをご紹介します！以下は基本的なチキンカレーのレシピです。\n\n### 材料（4人分）\n- 鶏もも肉：400g（食べやすい大きさにカット）\n- 玉ねぎ：2個（みじん切り）\n- にんにく：2片（みじん切り）\n- 生姜：1片（みじん切り）\n- トマト：1個（ざく切り）\n- カレーパウダー：大さじ2\n- クミンシード：小さじ1（お好みで）\n- ココナッツミルク：200ml（お好みで）\n- サラダ油：大さじ2\n- 塩：適量\n- 黒胡椒：適量\n- 水：400ml\n- パクチー（飾り用）：適量\n\n### 作り方\n1. **下準備**：\n   - 鶏肉に塩と黒胡椒をふりかけて下味をつけておきます。\n\n2. **玉ねぎを炒める**：\n   - 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを加え、中火で透明になるまで炒めます。\n\n3. **香味野菜を加える**：\n   - にんにくと生姜を加え、香りが立つまでさらに炒めます。\n\n4. **鶏肉を加える**：\n   - 鶏肉を鍋に加え、表面が白くなるまで炒めます。\n\n5. **スパイスを加える**：\n   - カレーパウダーとクミンシードを加え、全体に絡めるように炒めます。\n\n6. **トマトと水を加える**：\n   - ざく切りにしたトマトを加え、少し煮込んでから水を加えます。沸騰したら、弱火にして20分ほど煮込みます。\n\n7. **ココナッツミルクを加える**（お好みで）：\n   - 煮込みが終わったら、ココナッツミルクを加え、さらに5分ほど煮ます。\n\n8. **味を調える**：\n   - 塩で味を調整し、必要に応じて黒胡椒を加えます。\n\n9. **盛り付け**：\n   - お皿に盛り付け、パクチーを散らして完成です。\n\n### 提供方法\nご飯やナンと一緒にお召し上がりください。お好みでヨーグルトやサラダを添えると、より美味しく楽しめます！\n\nぜひお試しください！', 'うどんのレシピをご紹介します。シンプルで美味しい「かけうどん」の作り方です。\n\n### 材料（2人分）\n- うどん（乾燥または生）: 2玉\n- だし汁: 600ml\n  - だしの素（顆粒）: 大さじ1（または本だしを使用）\n  - 水: 

### LCELの「|」でさまざまなRunnableを連結させる

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーの質問にステップバイステップで回答してください。"),
        ("human", "{question}")
    ]
)

cot_chain = cot_prompt | model | output_parser

summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ステップバイステップで考えた回答から結論だけ抽出してください"),
        ("human", "{text}")
    ]
)

summarize_chain = summarize_prompt | model | output_parser

cot_summarize_chain = cot_chain | summarize_chain
cot_summarize_chain.invoke({"question": "10 + 2 * 3"})

'16'

## 5.2 RunnableLambda--任意の関数をRunnableにする

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}")
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

from langchain_core.runnables import RunnableLambda

def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | output_parser | RunnableLambda(upper)

output = chain.invoke({"input": "hello"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


### chainデコレーターを使ったRunnableLambdaの実装

In [ ]:
from langchain_core.runnables import chain

@chain
def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | output_parser | upper

output = chain.invoke({"input": "hello"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


### RunnableLambdaへの自動変換

In [ ]:
def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | output_parser | upper

output = chain.invoke({"input": "hello"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


### Runnableの入力の型と出力の型に注意

In [ ]:
def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | StrOutputParser() | upper

output = chain.invoke({"input": "hello"})
print(output)


HELLO! HOW CAN I ASSIST YOU TODAY?


## 5.3 RunnableParallel--複数のRunnablewo並列につなげる

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見をください。"),
        ("human", "{topic}")
    ]
)

optimistic_chain = optimistic_prompt | model | output_parser

pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください。"),
        ("human", "{topic}")
    ]
)

pessimistic_chain = pessimistic_prompt | model | output_parser


import pprint
from langchain_core.runnables import RunnableParallel

paraller_chain = RunnableParallel(
  {
    "optimistic_opinion":optimistic_chain,
    "pessimistic_opinion  ":pessimistic_chain
  }
  )

output = paraller_chain.invoke({"topic": "AI"})
pprint.pprint(output)

{'optimistic_opinion': 'AIは本当に素晴らしい技術ですね！私たちの生活を便利にし、さまざまな分野での革新を促進しています。医療、教育、環境問題など、AIの力を借りてより良い未来を築くことができるのです。これからもAIが進化し、私たちの可能性を広げてくれることを楽しみにしています！',
 'pessimistic_opinion  ': 'AIは確かに便利なツールですが、その進化には多くの懸念が伴います。人間の仕事が奪われたり、プライバシーが侵害されたりするリスクが高まっています。さらに、AIの判断が誤っている場合、その影響は計り知れません。結局、私たちが作り出したものが、私たち自身を脅かす存在になる可能性があるのです。'}


### RunnableParallelの出力をRunnableの入力に連結する

In [ ]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的AIです。２つの意見をまとめてください。"),
        ("human", "楽観的意見:{optimistic_opinion}\n悲観的意見:{pessimistic_opinion}"),
    ]
)

synthesize_chain =  (
    RunnableParallel(
        {
            "optimistic_opinion":optimistic_chain,
            "pessimistic_opinion":pessimistic_chain,
        }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)


生成AIの進化については、楽観的な意見と悲観的な意見が存在します。楽観的な見方では、生成AIの技術が進むことで、私たちの生活が便利で豊かになり、クリエイティブな作業や問題解決の支援を通じて、AIが私たちのアイデアを実現するパートナーとして機能することが期待されています。また、教育や医療、エンターテインメントなどの分野での革新が進むことで、より多くの人々が新しい知識や体験にアクセスでき、社会全体が進化する可能性があるとされています。

一方で、悲観的な見方では、生成AIの進化には多くの懸念が伴います。技術の進化が仕事の喪失や情報の信頼性の低下を引き起こすリスクがあり、AIが生成するコンテンツの氾濫によって真実と虚偽の区別が難しくなることで、社会が混乱する可能性も指摘されています。このように、生成AIの進化は便利さをもたらす一方で、私たちの未来に対する脅威ともなり得るという両面が存在しています。


### RunnableParallelへの自動変換

In [ ]:
synthesize_chain =  (
    {
        "optimistic_opinion":optimistic_chain,
        "pessimistic_opinion":pessimistic_chain,
    }
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化については、楽観的な意見と悲観的な意見が存在します。楽観的な意見では、生成AIの技術が進むことで私たちの生活が便利で豊かになり、クリエイティブな作業や問題解決のパートナーとしての役割を果たすことが期待されています。この進化により新しい発見や革新が続き、未来が明るくなる可能性にワクワクする声が上がっています。

一方で、悲観的な意見では、生成AIの進化には多くの懸念が伴い、仕事の喪失や情報の信頼性の低下、オリジナリティの喪失といったリスクが指摘されています。便利さの裏には代償があり、技術の扱い方が未来に大きな影響を与えることが強調されています。結局、生成AIの進化はその利点とリスクの両面を持ち合わせており、私たちがどのようにこの技術を活用するかが重要であると言えるでしょう。


### RunnableLambdaとの組み合わせ--itemgetterを使う例

In [ ]:
from operator import itemgetter

topic_getter = itemgetter("topic")
topic = topic_getter({"topic": "生成AIの進化について"})
print(topic)



生成AIの進化について


In [ ]:
from operator import itemgetter

synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的AIです。{topic}について２つの意見をまとめてください。"),
        ("human", "楽観的意見:{optimistic_opinion}\n悲観的意見:{pessimistic_opinion}"),
    ]
)

synthesize_chain =  (
    {
        "optimistic_opinion":optimistic_chain,
        "pessimistic_opinion":pessimistic_chain,
        "topic":itemgetter("topic"),
    }
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化に関する意見は、楽観的な見方と悲観的な見方の二つに分かれます。

**楽観的意見**では、生成AIの進化が私たちの生活を便利で豊かにする可能性を強調しています。技術の進歩により、クリエイティブな作業や問題解決の支援が行われ、AIがアイデアを実現するためのパートナーとして機能することが期待されています。また、教育、医療、エンターテインメントなどの分野での革新が進むことで、より多くの人々が新しい知識や体験にアクセスでき、社会全体の進化が促進されることにワクワク感を抱いています。将来的には、AIと人間が協力してより良い世界を築く可能性があると考えています。

**悲観的意見**では、生成AIの進化に伴うリスクや懸念が指摘されています。技術の進化が仕事の喪失や情報の信頼性の低下を引き起こす可能性があり、AIが生成するコンテンツが人間の創造性を脅かすことも懸念されています。また、AIの影響が私たちの思考や感情に悪影響を及ぼす可能性があり、便利さの裏には不安がつきまとい、未来が不透明になるのではないかという懸念が表明されています。

このように、生成AIの進化には明るい未来を期待する声と、リスクを警戒する声が共存しています。


## 5.4 RunnablePassthrough--入力をそのまま出力する

In [ ]:
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")


In [ ]:
!pip install tavily-python==0.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community==0.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.10
    Uninstalling langsmith-0.2.10:
      Successfully uninstalled langsmith-0.2.10


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈:"""
{context}
"""

質問:{question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

#output = chain.invoke("東京の今日の天気は？")
output = chain.invoke("今日のニュースは？")
print(output)

今日のニュースには、以下のトピックスが含まれています：

1. 自転車で信号無視の10歳児が車と衝突し、「過失割合100％」と認定され、保護者に賠償リスクがあるというニュース。
2. 斎藤元彦氏に対する証人尋問への出頭要請が決定されたこと。
3. 立花氏が脅されたと訴え、ネットの暴力についての議論が起きていること。
4. 埼玉栄高校で無免許運転が常態化していた可能性があるという報道。
5. 避妊目的の子宮内避妊用具の挿入手術に関する女性たちの訴え。

これらのニュースは、政治、社会、教育など多岐にわたる内容が含まれています。


### assign--RunnableParallelの出力に値を追加する

In [ ]:
import pprint

chain = {
    "question": RunnablePassthrough(),
    "context": retriever
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("東京の今日の天気は？")
pprint.pprint(output)

{'answer': '東京の今日、11月19日(火)の天気は「晴時々曇」で、最高気温は13℃、最低気温は8℃です。降水確率は0％で、北の風が吹く予報です。',
 'context': [Document(metadata={'title': '東京（東京）の天気 - Yahoo!天気・災害', 'source': 'https://weather.yahoo.co.jp/weather/jp/13/4410.html', 'score': 0.8437023, 'images': []}, page_content='パーソナル天気 現在位置： 天気・災害トップ > 関東・信越 > 東京都 > 東京（東京） 今日明日の天気 2024年11月19日 4時00分発表 11月19日(火) 晴時々曇 13℃[-7] 8℃[-3] 時間  0-6 6-12    12-18   18-24 降水  0％  0％  0％  0％ 風： 北の風後北東の風２３区西部では北の風やや強く 波： 1.5メートル後0.5メートル 11月20日(水) 曇のち雨 12℃[-1] 7℃[-1] 時間  0-6 6-12    12-18   18-24 降水  0％  10％ 30％ 60％ 風： 北西の風後東の風 波： 0.5メートル 東京地方の警報・注意報 注意報 強風、波浪 週間天気 2024年11月19日 2時00分発表 日付 11月21日 (木) 11月22日 (金) 11月23日 (土) 11月24日 (日) 11月25日 (月) 11月26日 (火) 天気 雨のち晴   晴時々曇   晴時々曇   晴時々曇   晴時々曇   曇一時雨 気温（℃）   14 降水 確率（％）   60  0   0   0   10  50 2024年11月19日 4時00分 発表 (C) Mapbox (C) OpenStreetMap (C) LY Corporation Yahoo!地図ガイドライン (C) Mapbox (C) OpenStreetMap (C) LY Corporation Yahoo!地図ガイドライン 11/18(月)18時\u3000師走の寒さ\u3000今季一番低く\u3000北日本中心に積雪増加も Copyright (C) 2024 Weather Map Co

In [ ]:
chain = (
    RunnableParallel(
      {
        "question": RunnablePassthrough(),
        "context": retriever
      }
  )
  .assign(answer=prompt | model | StrOutputParser())
  #.pick(["context", "answer"])
)
output = chain.invoke("東京の今日の天気は？")
pprint.pprint(output)

{'answer': '東京の今日、10月27日（日）の天気は、最高気温24℃、最低気温17℃で、降水確率は50%です。',
 'context': [Document(metadata={'title': '東京都の天気（今日・明日の天気） | @nifty天気予報', 'source': 'https://weather.nifty.com/prefecture/13', 'score': 0.86052185, 'images': []}, page_content='東京都の天気（今日・明日の天気） | @nifty天気予報 @niftyトップ 今日・明日の天気 サイトマップ MY天気 地震情報台風情報警報・注意報花粉情報洗濯指数傘指数衛星画像天気図 スポーツレジャー観光地公共施設 全国の天気 東京都 台風20号、21号発生中 東京都の天気 今日の天気10月27日(日) 明日の天気10月28日(月) 東京  24/17 50% 大島  23/19 50% 八丈島  25/21 40% 父島  29/26 30% 港区 目黒区 大田区 世田谷区 墨田区 江東区 荒川区 足立区 葛飾区 日野市 多摩市 稲城市 多摩北部 三鷹市 昭島市 小平市 東村山市 東大和市 清瀬市 青梅市 大島 八丈島 東京  21/17 70% 大島  21/21 70% 八丈島  26/23 70% 父島  29/26 40% 港区 目黒区 大田区 世田谷区 墨田区 江東区 荒川区 足立区 葛飾区 日野市 多摩市 稲城市 多摩北部 三鷹市 昭島市 東村山市 東大和市 清瀬市 青梅市 大島 八丈島 栃木県 群馬県 東京都 長崎県 熊本県 大分県 宮崎県 沖縄県 お天気豆知識 ### 音速 秋が深まり気温も下がってくると、夏、元気に鳴いていた虫たちの声も次第に聞こえなくなります。特に夜などはひっそりとした静けさを感じることでしょう。皆さんは、気温の変化が音の伝わる速さにも影響する、ということを知っていますか。空気中では、気温が高いほど音は速く伝わります。また、気圧も音速に影響を与えますが、無視できるほど小さなものです。一方、地上のようには電波の伝わらない海中においては、音は空気中よりも速く伝わり、重要な通信手段となっています。水中では、水温が高いほど音速は速くなり、水圧が高くても速く伝わる